# Polytop FDB Access: Vertical Profile Feature

## Installation
See instruction in [Polytope installation](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1)

**Configuring Access to Polytope**

In [ ]:
import os

#ECMWF polytope credentials
# leave empty for ICON-CSCS polytope
os.environ["POLYTOPE_USER_KEY"] = ""
os.environ["POLYTOPE_USER_EMAIL"] = ""

#ICON-CSCS polytope credentials
# leave empty for ecmwf polytope
os.environ["POLYTOPE_USER_KEY"] = "xxxx"
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"
os.environ["POLYTOPE_LOG_LEVEL"] = "INFO"

**Selecting date,time of the forecast**

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 6
rounded_hour = (past_time.hour // 6) * 6
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time

**Define and rotate the desired point for the vertical profile**
> **IMPORTANT**: The function `transform_point()` expects first longitude and then latitude.

In [66]:
import cartopy.crs as ccrs

# point for vertical profile
geo_point = [8, 47]

# South pole rotation of lon=10, latitude=-43
rotated_crs = ccrs.RotatedPole(
    pole_longitude=190, pole_latitude=43
)

# Convert a point from geographic to rotated coordinates
geo_crs = ccrs.PlateCarree()
rotated_point = rotated_crs.transform_point(geo_point[0], geo_point[1], geo_crs)

**Define the request**

In [67]:
from meteodatalab import mars

request = mars.Request(
    param="T",
    date=date,
    time=time,
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.PRESSURE_LEVEL,
    number=1,
    step=0,
    levelist=range(700,1000,50),
    feature={
        "type" : "verticalprofile",
        "points" : [rotated_point],
        "axes" : "levelist"
    }
)

**Retrieve the data**

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mchgj",
    request.to_polytope(),
    stream=False
).to_xarray()

**Plot the data**

In [70]:
from earthkit.plots.interactive import Chart

chart = Chart()
chart.line(ds["t"], y="levelist")
chart.fig.update_layout(yaxis1={"title": "hPa"})

chart.title(f"Vertical Profile: Temperature (date={date}, time={time})")

chart.show()